In [2]:
import pandas as pd
import numpy as np
from datetime import datetime 
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.externals import joblib

C:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
Line_46A = pd.read_csv('46A.csv',sep=',')

In [ ]:
Line_46A

In [ ]:
# Line_145 = pd.read_csv('145.csv',sep=',')
# Line_145

In [ ]:
# Line_47 = pd.read_csv('47.csv',sep=',')
# Line_47 

In [ ]:
weather = pd.read_excel('2018_historic_weather.xlsx',sheet_name='Sheet1')
weather

In [ ]:
weather.isnull().sum()

In [ ]:
weather = weather.drop(['sea_level','grnd_level','rain_1h','rain_3h','snow_1h','snow_3h'],axis=1)
weather

In [ ]:
weather.nunique()

In [ ]:
weather = weather.drop(['timezone','city_name','lat','lon','weather_description','weather_icon'],axis=1)
weather

In [ ]:
Line_46A.DAYOFSERVICE[0]

In [ ]:
weather.dt_iso[0]

In [ ]:
weather.dt_iso[0]==Line_46A.DAYOFSERVICE[0]

In [ ]:
print(datetime.fromtimestamp(weather.dt[0]))

In [ ]:
# Line_46A_new = pd.concat([Line_46A,Line_46A.DIRECTION.rename('DAYTIME')],axis=1)
# Line_46A_new
new = Line_46A.DIRECTION.rename('DAYTIME')
Line_46A

In [ ]:
# pd.to_datetime(Line_46A.DAYOFSERVICE[0],format='%d-%m-%Y %H:%M:%S')
for i in range(Line_46A.shape[0]):
    d = datetime.strptime(Line_46A.DAYOFSERVICE[i],'%d-%b-%y %H:%M:%S')
    hour = Line_46A.ACTUALTIME_ARR[i]//3600
    d = d + timedelta(hours=int(hour))
    new[i] = d
    print(i,end=' ')

In [ ]:
Line_46A_new = pd.concat([Line_46A,new.rename('DAYTIME')],axis = 1)
Line_46A_new

In [ ]:
Line_46A_new = Line_46A_new.drop(['DAYOFSERVICE','TRIPID'],axis=1)
Line_46A_new

In [ ]:
Line_46A_new.DAYTIME = pd.to_datetime(new)
Line_46A_new.dtypes

In [ ]:
new = weather.dt
for i in range(new.shape[0]):
    new[i] = datetime.fromtimestamp(weather.dt[i])

In [ ]:
weather = pd.concat([weather,new.rename('DAYTIME')],axis=1)
weather

In [ ]:
weather = weather.drop(['dt','dt_iso'],axis=1)
weather

In [ ]:

for i in range(weather.shape[0]):
    if weather.weather_main[i] == 'Clear':
        weather.weather_main[i]=0
    elif weather.weather_main[i] == 'Mist':
        weather.weather_main[i]=1
    elif weather.weather_main[i] == 'Fog':
        weather.weather_main[i]=3
    elif weather.weather_main[i] == 'Smoke':
        weather.weather_main[i]=4
    elif weather.weather_main[i] == 'Clouds':
        weather.weather_main[i]=5
    elif weather.weather_main[i] == 'Drizzle':
        weather.weather_main[i]=6
    elif weather.weather_main[i] == 'Rain':
        weather.weather_main[i]=7
    elif weather.weather_main[i] == 'Snow':
        weather.weather_main[i]=8
weather

In [ ]:
weather.DAYTIME = pd.to_datetime(weather.DAYTIME)
weather.to_csv('Weather_ReadytoMerge.csv',header=True,index=False)

In [ ]:
new = weather.temp
for i in range(weather.shape[0]):
    new[i] = weather.DAYTIME[i].dayofweek
weather = pd.concat([weather,new.rename('DAYOFWEEK')],axis=1)
weather

In [5]:
# dataset = pd.merge(Line_46A_new,weather,on='DAYTIME')
# dataset.to_csv('dataset46A.csv',header=True,index = False)
dataset=pd.read_csv('Step_2/dataset_46A.csv',sep=',')
dataset.head()

,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,VEHICLEID,DIRECTION,DAYTIME,TIME,DATE,ORDER,temp,...,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,DAYOFWEEK
0,30,762,32393,2868383,1,2018-07-01 08:00:00,31800,2018-07-01 00:00:00,1,6.0,...,15.0,17.64,1015,88,1.54,0,79,803,5,6.0
1,15,334,31505,2868383,1,2018-07-01 08:00:00,31200,2018-07-01 00:00:00,1,6.0,...,15.0,17.64,1015,88,1.54,0,79,803,5,6.0
2,18,842,31820,2868383,1,2018-07-01 08:00:00,31800,2018-07-01 00:00:00,1,6.0,...,15.0,17.64,1015,88,1.54,0,79,803,5,6.0
3,21,847,31994,2868383,1,2018-07-01 08:00:00,31800,2018-07-01 00:00:00,1,6.0,...,15.0,17.64,1015,88,1.54,0,79,803,5,6.0
4,22,848,32051,2868383,1,2018-07-01 08:00:00,31800,2018-07-01 00:00:00,1,6.0,...,15.0,17.64,1015,88,1.54,0,79,803,5,6.0


In [ ]:
dataset[train_features].nunique()

In [ ]:
numeric_features = ['PROGRNUMBER','temp','feels_like',
       'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed',
       'wind_deg', 'clouds_all']
categorical_features = ['STOPPOINTID','DIRECTION','DAYOFWEEK','weather_main','weather_id',]

In [ ]:
# Correlation matrix using code found on https://stanford.edu/~mwaskom/software/seaborn/examples/many_pairwise_correlations.html 
sns.set(style="white")  

# Calculate correlation of all pairs of continuous features 
corr = Xtrain[numeric_features].corr()  

# Generate a mask for the upper triangle 
mask = np.zeros_like(corr, dtype=np.bool) 
mask[np.triu_indices_from(mask)] = True  

# Set up the matplotlib figure 
f, ax = plt.subplots(figsize=(20, 20))  

# Generate a custom colormap - blue and red 
cmap = sns.diverging_palette(220, 10, as_cmap=True)  

# Draw the heatmap with the mask and correct aspect ratio 
sns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,             
            square=True, xticklabels=True, yticklabels=True,             
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax) 

plt.yticks(rotation = 0) 
plt.xticks(rotation = 90)

In [ ]:
for i in numeric_features:
    dataset[i].plot(kind='box',figsize = (5,5))
    plt.show()

In [ ]:
dataset[categorical_features]

In [2]:
dataset = pd.read_csv('dataset_47.csv',sep=',')
dataset

,PROGRNUMBER,STOPPOINTID,ACTUALTIME_ARR,VEHICLEID,DIRECTION,DAYTIME,TIME,DATE,ORDER,LINEID,...,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,DAYOFWEEK
0,1,7662,33858,2868377,1,2018-07-01 09:00:00,33600,2018-07-01 00:00:00,1,47.0,...,15.97,18.86,1016,77,2.06,360,86,804,5,6.0
1,1,5013,34269,2868327,2,2018-07-01 09:00:00,34200,2018-07-01 00:00:00,1,47.0,...,15.97,18.86,1016,77,2.06,360,86,804,5,6.0
2,2,7564,33858,2868377,1,2018-07-01 09:00:00,33600,2018-07-01 00:00:00,1,47.0,...,15.97,18.86,1016,77,2.06,360,86,804,5,6.0
3,2,5014,34312,2868327,2,2018-07-01 09:00:00,34200,2018-07-01 00:00:00,1,47.0,...,15.97,18.86,1016,77,2.06,360,86,804,5,6.0
4,4,350,34502,2868377,1,2018-07-01 09:00:00,34200,2018-07-01 00:00:00,1,47.0,...,15.97,18.86,1016,77,2.06,360,86,804,5,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167070,10,4984,24188,1000154,2,2018-10-01 06:00:00,24000,2018-10-01 00:00:00,1,47.0,...,4.00,8.20,1028,93,2.10,250,20,801,5,0.0
167071,3,7291,23557,1000154,2,2018-10-01 06:00:00,23400,2018-10-01 00:00:00,1,47.0,...,4.00,8.20,1028,93,2.10,250,20,801,5,0.0
167072,5,3486,23712,1000154,2,2018-10-01 06:00:00,23400,2018-10-01 00:00:00,1,47.0,...,4.00,8.20,1028,93,2.10,250,20,801,5,0.0
167073,6,3487,23748,1000154,2,2018-10-01 06:00:00,23400,2018-10-01 00:00:00,1,47.0,...,4.00,8.20,1028,93,2.10,250,20,801,5,0.0


In [14]:
train_features = ['STOPPOINTID','DAYOFWEEK','PROGRNUMBER','ORDER','temp','feels_like',
        'pressure', 'humidity', 'wind_speed','wind_deg', 'clouds_all', 'weather_id', 'weather_main']
target_features = ['ACTUALTIME_ARR']

In [15]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(dataset[train_features],dataset[target_features],test_size = 0.3)

In [ ]:
rfc = RandomForestRegressor(
                            random_state=0
                            )
rfc = rfc.fit(Xtrain,Ytrain)
score = rfc.score(Xtest,Ytest)
score

In [8]:
def printMetrics(testActualVal, predictions): 
    #classification evaluation measures
    print('\n==============================================================================') 
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions)) 
    #print("MSE: ", metrics.mean_squared_error(testActualVal, predictions)) 
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5) 
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [ ]:
test_prediction = rfc.predict(Xtest)
printMetrics(Ytest,test_prediction)

In [ ]:
# joblib.dump(rfc,'Model_for_46A')

In [16]:
lnr = LinearRegression().fit(Xtrain,Ytrain)
score = lnr.score(Xtest,Ytest)
score

0.9219539149400389

In [17]:
test_prediction = lnr.predict(Xtest)
printMetrics(Ytest,test_prediction)


MAE:  3599.8691591802835
RMSE:  4753.235857998325
R2:  0.9219539149400388


In [18]:
polynomial = PolynomialFeatures(degree=2)
x_transformed = polynomial.fit_transform(Xtrain)

ply = LinearRegression()
ply = ply.fit(x_transformed,Ytrain)

In [19]:
ply.score(polynomial.fit_transform(Xtest),Ytest)

0.9678831858586648

In [20]:
test_prediction = ply.predict(polynomial.fit_transform(Xtest))
printMetrics(Ytest,test_prediction)


MAE:  2083.3811674676444
RMSE:  3049.158726602011
R2:  0.9678831858586648
